In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from genericpath import exists
import statistics
import scipy.stats as st

In [7]:
def process_df_for_gantt(df: pd):

    worker_dict = {}

    for row in df.itertuples(index=True, name="Pandas"):
        task_id = str(row.task_id)
        start = row.start_time
        end = row.end_time
        duration = end - start
        host_ip = row.host_ip
        pid = row.pid
        task = row.file_name + "/" + row.column_name
        worker_dict[task_id] = {
            "start": start,
            "end": end,
            "duration": duration,
            "host_ip": host_ip,
            "pid": pid,
            "task": task,
        }

    return worker_dict

In [8]:
def analyze_signle_local_logs_file(
    logs_file_path_name: str = None,
    initial_process_time: float = 0,
    total_process_time: float = 0,
    instance_number:int = 0 ,
    num_unfinished_tasks: int = 0,
    code_templates:str = None,

) -> dict:

    if exists(logs_file_path_name) is False:
        print(f"{logs_file_path_name} does not exist")
        return
    df = pd.read_csv(logs_file_path_name)

    error_task = df[(df["alert_type"] == "SYSTEM_ERROR")]
    num_error_task = len(error_task)

    worker_dict = process_df_for_gantt(df)

    shortest_task = ""
    longest_task = ""
    min_duration = float("inf")
    max_duration = 0
    tasks_durtaion_sum = 0
    average_task_duration = 0
    total_tasks = 0
    min_start = float("inf")
    max_end = 0
    duration = 0
    task_duration_in_parallelism = 0
    efficiency = 0
    for key, value in worker_dict.items():
        # logger.info(f"ip {value['host_ip']} ")
        # logger.info(f"ip {value} ")
        if ("start" in value) is False:
            # logger.warning(f"missing 'start' key in task {key}")
            print(f"missing 'start' key in task {key}")
            continue
        if ("end" in value) is False:
            # logger.warning(f"missing 'end' key in task {key}")
            print(f"missing 'end' key in task {key}")
            continue
        start = float(value["start"])
        end = float(value["end"])
        duration = value["duration"]
        task = value["task"]
        if task == "loop_tasks_status_task":
            continue
        tasks_durtaion_sum += duration
        if start < min_start:
            min_start = start
        if end > max_end:
            max_end = end

        if duration < min_duration:
            min_duration = duration
            shortest_task = task
        if duration > max_duration:
            max_duration = duration
            longest_task = task
        total_tasks += 1
    task_duration_in_parallelism = max_end - min_start
    if total_tasks > 0:
        average_task_duration = tasks_durtaion_sum / total_tasks
    else:
        average_task_duration = tasks_durtaion_sum


    ip_info = dict()
    
    # --------------------------
    # calcuate effeciency factor
    # --------------------------
    # step 1 , accumulate the process time of each host_ip/pid
    for key, value in worker_dict.items():
        _duration = float(value["duration"])
        # _start = float(value["start"])
        # _end = float(value["end"])
        _host_ip = value["host_ip"]
        _pid = value["pid"]
        
        # ip = str(_host_ip) + "/" + str(_pid)
        ip= str(_host_ip)
        if ip in ip_info:
            ip_info[ip][key] = value
        else:
            ip_info[ip] = {
                key: value
            }
    # print("===========\n")
    
    for key , value in ip_info.items():
        _task_min_start  = float("inf")
        _task_max_end = 0
        _accu_duration = 0
        for taskid, taskvalue in value.items():
            _start = taskvalue['start']
            _end = taskvalue['end']
            _accu_duration += taskvalue['duration']
            if _task_min_start >_start:
                _task_min_start = _start
            if _task_max_end < _end:
                _task_max_end = _end
        ip_info[key]['min_start'] = float(_task_min_start)
        ip_info[key]['max_end'] = float(_task_max_end)
        ip_info[key]['task_accumulate_durations'] =float(_accu_duration)
        ip_info[key]['ip_max_min_duration'] = float(2*(_task_max_end -_task_min_start))
        ip_info[key]['ip_efficiency'] = float(_accu_duration*100)/float(2*(_task_max_end -_task_min_start))

    file_accu_efficicency = 0 
    time_efficiency = 0 
    for key , value in ip_info.items():
        file_accu_efficicency +=  ip_info[key]['ip_efficiency'] 
    file_accu_efficicency = file_accu_efficicency/len(ip_info)

    time_efficiency = tasks_durtaion_sum *100 /(task_duration_in_parallelism * len(ip_info)*2)
    print(f"time_efficiency : {time_efficiency}")
    messages_rate = 0
    if task_duration_in_parallelism > 0: 
        messages_rate = 3*total_tasks / task_duration_in_parallelism
    # # normalizing_process_time = task_duration_in_parallelism / (total_tasks*average_task_duration) 
    performance_dict = {
        "file": logs_file_path_name,
        "instance_number": instance_number,
        "code_templates": code_templates,
        "total_tasks": total_tasks,
        "average_task_duration": round(average_task_duration, 2),
        "min_duration": round(min_duration, 2),
        "max_duration": round(max_duration, 2),
        "num_error_task": num_error_task,
        "longest_task": longest_task,
        "shortest_task": shortest_task,
        "num_unfinished_tasks": num_unfinished_tasks,
        "task_duration_in_parallelism": round(task_duration_in_parallelism, 2),
        "tasks_durtaion_sum": round(tasks_durtaion_sum, 2),
        "initial_process_time": round(initial_process_time, 2),
        "total_process_time": round(total_process_time, 2),
        "time_efficiency": round(time_efficiency, 2),
        "resource_efficiency": round(file_accu_efficicency, 2),
        "messages_rate": round(messages_rate, 2),
        # "normalizing_process_time": round(normalizing_process_time,2),
        # "optimal_normalizing_process_time": round(1/instance_number,2),
    }
    return performance_dict

In [9]:
def get_mean_std_and_upper_lower_of_confidence_interval(data:list = None,alpha:float = 0.95 ) -> dict:
    info = {}
    mean = round(
            statistics.mean(data[:]), 2
        )
    std = round(
        statistics.stdev(data[:]), 2
    )
    lower_confidence_interval, upper_confidence_interval= st.t.interval(alpha=alpha, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))
    info = {
        "mean":mean,
        "std":std,
        "lower_confidence_interval":round(mean-lower_confidence_interval,2),
        "upper_confidence_interval":round(upper_confidence_interval-mean,2)

    }
    return info

In [10]:
logs_folder_list = [
    './datasets/gridlabd-mix/gridlabd-2',
     './datasets/gridlabd-mix/gridlabd-4',
     './datasets/gridlabd-mix/gridlabd-8',
     './datasets/gridlabd-mix/gridlabd-16',
    './datasets/gridlabd-mix/gridlabd-24',
     './datasets/gridlabd-mix/gridlabd-32',]
# logs_folder_list = [
#     './datasets/gridlabd-NY/gridlabdNY-1',
#     './datasets/gridlabd-NY/gridlabdNY-2',
#     './datasets/gridlabd-NY/gridlabdNY-4',
#     './datasets/gridlabd-NY/gridlabdNY-8',
#     './datasets/gridlabd-NY/gridlabdNY-12',
#     './datasets/gridlabd-NY/gridlabdNY-16',
#     './datasets/gridlabd-NY/gridlabdNY-24',
#     './datasets/gridlabd-NY/gridlabdNY-32',]
# logs_folder_list = [
#     './datasets/gridlabd-mixed/gridlabdsorted-8',
#     './datasets/gridlabd-mixed/gridlabdsorted-12',
#     './datasets/gridlabd-mixed/gridlabdsorted-16',
#     './datasets/gridlabd-mixed/gridlabdsorted-24',
#     './datasets/gridlabd-mixed/gridlabdsorted-32',]
index = 0
logs_files_list = []

# process log file and convert to performance dict
performance = {}
number_of_message_per_task = 3
for _folder in logs_folder_list:
    index = 0 
    base, path = os.path.split(_folder)
    # print(base, path)
    code_templates, number_instance = path.split("-")
    # print(code_templates,number_instance)
    _per_list = []
    _task_duration_in_parallelism_list = []
    _time_efficiency_list = []
    _resource_efficiency_list = []
    _average_task_duration_list = []
    _messages_rate_list = []
    _normalizing_process_time_list = []
    # _optimal_normalizing_process_time_list = []
    total_tasks = 1
    for _file in os.listdir(_folder):
        filename,extension = os.path.splitext(_file)
        if extension == ".csv":
            _name_comp = filename.split("-")
            
            if len(_name_comp) > 1 and _name_comp[0] == "logs": 
                logs_file_path_name  = _folder +"/"+_file
    
                per_dict = analyze_signle_local_logs_file(
                    logs_file_path_name=logs_file_path_name,
                    instance_number = number_instance,
                    initial_process_time=0,
                    total_process_time=0,
                    num_unfinished_tasks=0,
                    code_templates = code_templates,
                )

                _task_duration_in_parallelism_list.append(per_dict['task_duration_in_parallelism'])
                _time_efficiency_list.append(per_dict['time_efficiency'])
                _resource_efficiency_list.append(per_dict['resource_efficiency'])
                _average_task_duration_list.append(per_dict['average_task_duration'])
                _messages_rate_list.append(per_dict['messages_rate'])

                total_tasks = per_dict['total_tasks']
                index += 1

    duration_in_parallelism_dict = get_mean_std_and_upper_lower_of_confidence_interval(_task_duration_in_parallelism_list)
    average_task_duration_dict = get_mean_std_and_upper_lower_of_confidence_interval(_average_task_duration_list)
    time_effeciencyFactor_dict =  get_mean_std_and_upper_lower_of_confidence_interval(_time_efficiency_list)
    resource_effeciencyFactor_dict =  get_mean_std_and_upper_lower_of_confidence_interval(_resource_efficiency_list)
    messages_rate_dict = get_mean_std_and_upper_lower_of_confidence_interval(_messages_rate_list)


    _info = {
            'code_templates':code_templates,
            'number_instance':number_instance,
            "duration_in_parallelism":duration_in_parallelism_dict,
            "time_efficiency":time_effeciencyFactor_dict,
            "resource_efficiency":resource_effeciencyFactor_dict,
            "average_task_duration":average_task_duration_dict,
            "messages_rate":messages_rate_dict,
    }

    performance[path] = _info

time_efficiency : 82.27745076770174
time_efficiency : 77.87172891429067
time_efficiency : 81.18798096107221
time_efficiency : 85.82951993324436
time_efficiency : 82.82728381443997
time_efficiency : 81.81311560879983
time_efficiency : 72.59176374822611
time_efficiency : 67.3789286379994
time_efficiency : 65.99094033961912
time_efficiency : 67.06136392325291
time_efficiency : 72.93623065826196
time_efficiency : 71.21353954680326
time_efficiency : 28.416524628666473
time_efficiency : 28.718837230263738
time_efficiency : 28.146477908544426
time_efficiency : 28.41757645074563
time_efficiency : 28.425339017293126
time_efficiency : 28.673225395161193


FileNotFoundError: [Errno 2] No such file or directory: './datasets/gridlabd-mix/gridlabd-16'

In [ ]:
# Get x axis
X_axis = []
for _folder in logs_folder_list:
    base, path = os.path.split(_folder)
    # print(base, path)
    code_templates, number_instance = path.split("-")
    X_axis.append(int(number_instance)*2)

In [ ]:
# Get Y 
mean_of_task_duration_in_parallelism_list = []
lower_confidence_interval_of_task_duration_in_parallelism = []
upper_confidence_interval_of_task_duration_in_parallelism = []

mean_of_time_efficiency_list = []
lower_confidence_interval_of_time_efficiency_list = []
upper_confidence_interval_of_time_efficiency_list= []

mean_of_resource_efficiency_list = []
lower_confidence_interval_of_resource_efficiency_list = []
upper_confidence_interval_of_resource_efficiency_list= []

mean_of_average_task_duration_list = []
lower_confidence_interval_of_average_task_duration_list  = []
upper_confidence_interval_of_average_task_duration_list =  []

mean_of_messages_rate_list = []
lower_confidence_interval_of_messages_rate_list = []
upper_confidence_interval_of_messages_rate_list= []

# mean_of_normalizing_process_time = []
# lower_confidence_mean_of_normalizing_process_time = []
# upper_confidence_mean_of_normalizing_process_time= []
optimal_normalizing_process_time_list = []

for _folder in logs_folder_list:
    base, path = os.path.split(_folder)
    # print(base, path)
    code_templates, number_instance = path.split("-")
    info = performance[path]
    
 
    # print(f'----------> {path}')
    # print(f"info: {info}")
    mean_of_task_duration_in_parallelism_list.append(info['duration_in_parallelism']['mean'])
    lower_confidence_interval_of_task_duration_in_parallelism.append(info['duration_in_parallelism']['lower_confidence_interval'])
    upper_confidence_interval_of_task_duration_in_parallelism.append(info['duration_in_parallelism']['upper_confidence_interval'])

    # time efficiency
    mean_of_time_efficiency_list.append(info['time_efficiency']['mean'])
    lower_confidence_interval_of_time_efficiency_list.append(info['time_efficiency']['lower_confidence_interval'])
    upper_confidence_interval_of_time_efficiency_list.append(info['time_efficiency']['upper_confidence_interval'])

    # resource efficiency
    mean_of_resource_efficiency_list.append(info['resource_efficiency']['mean'])
    lower_confidence_interval_of_resource_efficiency_list.append(info['resource_efficiency']['lower_confidence_interval'])
    upper_confidence_interval_of_resource_efficiency_list.append(info['resource_efficiency']['upper_confidence_interval'])

    
    mean_of_average_task_duration_list.append(performance[path]['average_task_duration']['mean'])
    lower_confidence_interval_of_average_task_duration_list.append(performance[path]['average_task_duration']['lower_confidence_interval'])
    upper_confidence_interval_of_average_task_duration_list.append(performance[path]['average_task_duration']['upper_confidence_interval'])

    
    mean_of_messages_rate_list.append(performance[path]['messages_rate']['mean'])
    lower_confidence_interval_of_messages_rate_list.append(performance[path]['messages_rate']['lower_confidence_interval'])
    upper_confidence_interval_of_messages_rate_list.append(performance[path]['messages_rate']['upper_confidence_interval'])

    optimal_normalizing_process_time = round(1/(int(number_instance)*2),4)
    optimal_normalizing_process_time_list.append(optimal_normalizing_process_time)


acutal_normalizing_process_time_list = []

for index, task_d in enumerate(mean_of_task_duration_in_parallelism_list):
    average_task_duration = mean_of_average_task_duration_list[index]
    # print(task_d,total_tasks,average_task_duration)
    acutal_normalizing_process_time = task_d/ (total_tasks*average_task_duration)
    acutal_normalizing_process_time_list.append(round(acutal_normalizing_process_time,4))


In [ ]:


fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex=False, sharey=False)
fig.set_size_inches(18.5, 10.5)
fig.set_dpi(100)
fig.suptitle("Gridlabd mix files(PG&E,IEEE, taxonomy) tasks mixed file",color="black")
datax = [ 10**i for i in range(2,64)]

# Plot figure 1

ax1.set_title('Normalizing process time', color="black")
ax1.set_xscale('log', base=2)
ax1.set_yscale('log', base=2)
ax1.set(title='Normalizing process time')
ax1.plot(X_axis,optimal_normalizing_process_time_list, 'b',marker = '+',label='Optimal time')
ax1.plot(X_axis,acutal_normalizing_process_time_list, 'r',marker = 'o',label='Actual time')
ax1.legend()
ax1.set(xlabel='vCPUs', ylabel='Normalizing process time')
ax1.xaxis.label.set_color('black')
ax1.yaxis.label.set_color('black')
ax1.tick_params(axis='x', colors='black')
ax1.tick_params(axis='y', colors='black')


# Plot figure 2
ax2.set_title('Efficiency and Total process time vs vCPUs', color="black")
ax2a = ax2.twinx()

ax2.plot(X_axis,mean_of_time_efficiency_list, 'b',marker = '+',label='Time efficiency')
ax2.plot(X_axis,mean_of_resource_efficiency_list, 'g',marker = '+',label='Resource efficiency')

ax2a.plot(X_axis,mean_of_task_duration_in_parallelism_list, 'r',marker = 'o',label='Total process time')
ax2.legend(loc='upper right')
ax2a.legend(loc="lower left")
ax2.errorbar(X_axis, mean_of_time_efficiency_list, yerr=(lower_confidence_interval_of_time_efficiency_list, upper_confidence_interval_of_time_efficiency_list), zorder=0, fmt="none",color="black")

ax2.errorbar(X_axis, mean_of_resource_efficiency_list, yerr=(lower_confidence_interval_of_resource_efficiency_list, upper_confidence_interval_of_resource_efficiency_list), zorder=0, fmt="none",color="green")


ax2a.errorbar(X_axis, mean_of_task_duration_in_parallelism_list, yerr=(lower_confidence_interval_of_task_duration_in_parallelism, upper_confidence_interval_of_task_duration_in_parallelism) ,zorder=0, fmt="none",color="red")

ax2.set_title('Efficiency and total process time vs vCPUs')
# ax2a.legend("Total process time(s)",loc='upper right')
# ax2.legend("Efficiency",loc='lower left')
ax2.tick_params(axis='x', colors='black')
ax2.tick_params(axis='y', colors='black')
ax2a.tick_params(axis='y', colors='black')
ax2.xaxis.label.set_color('black')
ax2.yaxis.label.set_color('black')
ax2a.yaxis.label.set_color('black')
ax2.set(xlabel='vCPUs', ylabel='Efficiency (%)')
ax2a.set(xlabel='vCPUs', ylabel='Total process time (sec)')


# Plot figure 3
ax3.set_title('Average duration vs vCPUs', color="black")
ax3.errorbar(X_axis, mean_of_average_task_duration_list, yerr=(lower_confidence_interval_of_average_task_duration_list, upper_confidence_interval_of_average_task_duration_list) ,marker = 'o')
ax3.set_title('Average duration (sec)')
ax3.set(xlabel='vCPUs', ylabel='Average duration (sec)')
ax3.tick_params(axis='x', colors='black')
ax3.tick_params(axis='y', colors='black')
ax3.xaxis.label.set_color('black')
ax3.yaxis.label.set_color('black')

# Plot figure 4
ax4.set_title('Messages rate vs vCPUs', color="black")
ax4.errorbar(X_axis, mean_of_messages_rate_list, yerr=(lower_confidence_interval_of_messages_rate_list, upper_confidence_interval_of_messages_rate_list) ,marker = 'o')
ax4.set_title('Messages rate')
ax4.set(xlabel='vCPUs', ylabel='# messages per task * total tasks / # total durations (# message per sec)')
ax4.tick_params(axis='x', colors='black')
ax4.tick_params(axis='y', colors='black')
ax4.xaxis.label.set_color('black')
ax4.yaxis.label.set_color('black')
plt.savefig('data.png')  
plt.show()
plt.close()